# Build a BOW Sentiment Classifier

This demo is a notebook with references from [CMU CS11-711 Advanced NLP](http://phontron.com/class/anlp2024/),  that trains a sentiment classifier based on data. Specifically, it uses a bag-of-words to extract features, and the structured perceptron algorithm to train the classifier.

It will take in a text `X` and return a `label` of \"1\" if the sentiment of the text is positive, \"-1\" if the sentiment of the text is negative, and \"0\" if the sentiment of the text is neutral. You can test the accuracy of your classifier on the [Stanford Sentiment Treebank](http://nlp.stanford.edu/sentiment/index.html) by running the notebook all the way to end.

In [1]:
import random
import tqdm

In [ ]:
!wget https://github.com/neubig/anlp-code/raw/refs/heads/main/data/sst-sentiment-text-threeclass/train.txt
!wget https://github.com/neubig/anlp-code/raw/refs/heads/main/data/sst-sentiment-text-threeclass/dev.txt

## Feature Extraction

Feature extraction code, how do we get the features we use in training? By default we just use every word.

In [2]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

Also, initialize the feature weights to zero.

In [3]:
feature_weights = {}

## Data Reading

Read in the data from the training and dev (or finally test) sets

In [4]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [5]:
x_train, y_train = read_xy_data('train.txt')
x_dev, y_dev = read_xy_data('dev.txt')

In [6]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


## Inference Code

How we run the classifier.

In [7]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

## Training Code

Learn the weights of the classifier.

In [8]:
NUM_EPOCHS = 5
for epoch in range(1, NUM_EPOCHS+1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            for feature in features:
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]

Epoch 5: 100%|██████████| 8544/8544 [00:00<00:00, 347572.68it/s]


In [9]:
print(f"Total features {len(feature_weights.keys())}")
for i,(k,v) in enumerate(feature_weights.items()):
  print(f"{k}\t\t{v}")
  if i>20:
    break

Total features 13136
One		-2.0
of		1.0
those		4.0
exceedingly		1.0
rare		9.0
films		1.0
in		1.0
which		-3.0
the		0.0
talk		1.0
alone		-2.0
is		2.0
enough		0.0
to		-5.0
keep		6.0
us		4.0
involved		-1.0
.		-2.0
...		-2.0
an		0.0
incredibly		-2.0
heavy-handed		-3.0


## Evaluation Code

How we evaluate the classifier:

In [10]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [11]:
label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{1: 444, 0: 229, -1: 428}


In [12]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.757373595505618
Dev/test accuracy: 0.5885558583106267


## Error Analysis

An important part of improving any system is figuring out where it goes wrong. The following two functions allow you to randomly observe some mistaken examples, which may help you improve the classifier. Feel free to write more sophisticated methods for error analysis as well.

In [13]:
def find_errors(x_data, y_data):
    error_ids = []
    y_preds = []
    for i, (x, y) in enumerate(zip(x_data, y_data)):
        y_preds.append(run_classifier(extract_features(x)))
        if y != y_preds[-1]:
            error_ids.append(i)
    for _ in range(5):
        my_id = random.choice(error_ids)
        x, y, y_pred = x_data[my_id], y_data[my_id], y_preds[my_id]
        print(f'{x}\ntrue label: {y}\npredicted label: {y_pred}\n')

In [14]:
find_errors(x_dev, y_dev)

Jones ... does offer a brutal form of charisma .
true label: 1
predicted label: -1

A taut psychological thriller that does n't waste a moment of its two-hour running time .
true label: 1
predicted label: -1

While there 's something intrinsically funny about Sir Anthony Hopkins saying ` Get in the car , bitch , ' this Jerry Bruckheimer production has little else to offer
true label: 1
predicted label: -1

... an otherwise intense , twist-and-turn thriller that certainly should n't hurt talented young Gaghan 's resume .
true label: 1
predicted label: -1

may not have generated many sparks , but with his affection for Astoria and its people he has given his tale a warm glow .
true label: 0
predicted label: 1

